# TensorFlow Datasets

## Instalación de tf.datasets

Antes de comenzar, aprenderemos un poco sobre cómo obtener conjuntos de datos con la librería TFDS (TensorFlow Datasets).

Puedes consultar la documentación oficial [aquí](https://www.tensorflow.org/datasets/overview)

TFDS existe en dos paquetes:

- `pip install tensorflow-datasets`: la versión estable, lanzada cada pocos meses.
- `pip install tfds-nightly`: Lanzado todos los días, contiene las últimas versiones de los conjuntos de datos.

Este notebook usa tfds-nightly:

In [ ]:
!pip install -q tfds-nightly

In [ ]:
import matplotlib as plt
import numpy as np
import tensorflow as tf
import pandas

import tensorflow_datasets as tfds

## Conjuntos de datos disponibles

Todos los creadores de conjuntos de datos son subclase de `tfds.core.DatasetBuilder`. Para obtener la lista de constructores disponibles, use `tfds.list_builders()`

In [ ]:
tfds.list_builders()

## Cargar un conjunto de datos

### tfds.load

La forma más sencilla de cargar un conjunto de datos es tfds.load. Va a:
1. Descargar los datos y guardarlos como archivos `tfrecord`.
2. Cargar el `tfrecord` y crear el `tf.data.Dataset`.

In [ ]:
ds = tfds.load('mnist', split='train', shuffle_files=True)
assert isinstance(ds, tf.data.Dataset)
print(ds)

- `split=`: Qué división leer (por ejemplo `'train'`, `['train', 'test']`, `train[80%]`, ...).
- `shuffle_files=`: Controla si mezclar los archivos entre cada época (TFDS almacena grandes conjuntos de datos en varios archivos más pequeños).
- `data_dir=`: Ubicación donde se guarda el conjunto de datos (por defecto es `~/tensorflow_datasets/`).
`with_info=True`: Devuelve el `tfds.core.DatasetInfo` contiene los metadatos del conjunto de datos.
`download=False`: deshabilitar la descarga.

### tfds.builder

Una alternativa a utilizar tdfs.load es utilizar la API tfds.core.DatasetBuilder:

In [ ]:
builder = tfds.builder('mnist')
# 1. Create the tfrecord files (no-op if already exists)
builder.download_and_prepare()
# 2. Load the `tf.data.Dataset`
ds = builder.as_dataset(split='train', shuffle_files=True)
print(ds)

## Iterar sobre el conjunto de datos

### Como diccionario

Por defecto, el objeto `tf.data.Dataset` contiene un `diccionario` de `tf.Tensor`

In [ ]:
ds = tfds.load('mnist', split='train')
ds = ds.take(1)  # Only take a single example

for example in ds:  # example is `{'image': tf.Tensor, 'label': tf.Tensor}`
  print(list(example.keys()))
  image = example["image"]
  label = example["label"]
  print(image.shape, label)

### Como una tupla (as_supervised=True)

Utilizando `as_supervised=True`, puedes obtener una tupla `(features, label)` para datasets supervisados.

In [ ]:
ds = tfds.load('mnist', split='train', as_supervised=True)
ds = ds.take(1)

for image, label in ds:  # example is (image, label)
  print(image.shape, label)

### Como numpy (tfds.as_numpy)

In [ ]:
ds = tfds.load('mnist', split='train', as_supervised=True)
ds = ds.take(1)

for image, label in tfds.as_numpy(ds):
  print(type(image), type(label), label)

### Como batched tf.Tensor (batch_size=-1)

In [ ]:
image, label = tfds.as_numpy(tfds.load(
    'mnist',
    split='test',
    batch_size=-1,
    as_supervised=True,
))

print(type(image), image.shape)

Asegúrate de que tu dataset quepa en memoria, y que todos los ejemplos tengan la misma forma.

## Visualización

### tfds.as_dataframe

Los objetos `tf.data.Dataset` pueden ser convertidos a `pandas.DataFrame` con `tfds.as_dataframe` para ser visualizados en un notebook, por ejemplo.

- Añade `tfds.core.DatasetInfo` como segundo argumento de `tfds.as_dataframe` para visualizar imágenes, audios, textos, videos, ...
- Utiliza `ds.take(x)` para mostrar únicamente los primeros `x` ejemplos. `pandas.DataFrame` cargará todo el dataset en memoria y podría ser difícil de mostrar.

In [ ]:
ds, info = tfds.load('mnist', split='train', with_info=True)

tfds.as_dataframe(ds.take(4))

### tfds.show_examples

In [ ]:
ds, info = tfds.load('mnist', split='train', with_info=True)

fig = tfds.show_examples(ds, info)

## Descargar los datasets manualmente

Si algo falla y no puede descargar los datos con `tf.datasets`, siempre puede consultar las URL de descarga directa en este [catálogo](https://www.tensorflow.org/datasets/catalog/overview).